# 3.2.2 - Pré Processamento de Dados: Dataframe Faltas

In [25]:
#Importando Pandas, OS para leitura do arquivo e Matplotlib

import pandas as pd
import os
import matplotlib.pyplot as plt

os.chdir('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro')

In [26]:
#Importando o data set dos dados das partidas dos campeonatos brasileiros e os dados dos cartões amarelos

df_jogos=pd.read_csv('campeonato-brasileiro-full.csv',sep=',')
df_faltas=pd.read_csv('campeonato-brasileiro-estatisticas-full.csv',sep=',')


In [4]:
#Foi identificada uma falta de dados no Data Frame de estatisticas, onde os dados de cartões e faltas são registrados após
#o registro da partida 4987. Já no Data Frame de cartões é identificado que os registros são contabilizados a partir da 
# partida com o ID 4607. Essa diferença é basicamente de um campeonato inteiro sem dados que poderiam ser aproveitado
#por esse motivo iremos tratar o Data Frame de cartões para utilizar os dados a partir deste Data Frame.

#Inicio Campeonato 4985 - 6885

df_faltas=df_faltas[(df_faltas.partida_id>=4987) & (df_faltas.partida_id<=6885)]
df_faltas[(df_faltas.partida_id>=5500)]

In [27]:
#Identificando valores nulos ou zerados no Data Frame de faltas
#Primeiramente verificamos o tamanho do Data Frame
df_jogos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8025 entries, 0 to 8024
Data columns (total 16 columns):
ID                    8025 non-null int64
rodata                8025 non-null int64
data                  8025 non-null object
hora                  8025 non-null object
mandante              8025 non-null object
visitante             8025 non-null object
formacao_mandante     3051 non-null object
formacao_visitante    3051 non-null object
tecnico_mandante      3415 non-null object
tecnico_visitante     3415 non-null object
vencedor              8025 non-null object
arena                 8025 non-null object
mandante_Placar       8025 non-null int64
visitante_Placar      8025 non-null int64
mandante_Estado       8025 non-null object
visitante_Estado      8025 non-null object
dtypes: int64(4), object(12)
memory usage: 1003.2+ KB


In [28]:
#Idenficando valores nulos
df_faltas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16050 entries, 0 to 16049
Data columns (total 13 columns):
partida_id         16050 non-null int64
rodata             16050 non-null int64
clube              16050 non-null object
chutes             16050 non-null int64
chutes_no_alvo     16050 non-null int64
posse_de_bola      6838 non-null object
passes             16050 non-null int64
precisao_passes    6838 non-null object
faltas             16050 non-null int64
cartao_amarelo     16050 non-null int64
cartao_vermelho    16050 non-null int64
impedimentos       16050 non-null int64
escanteios         16050 non-null int64
dtypes: int64(10), object(3)
memory usage: 1.6+ MB


In [9]:
#Visualizando o Data Frame
df_faltas.head(100)

In [32]:
#Para identificar os registros que possuem os valores de cartões zerados de fato nas partidas será necessário um tratamento no
#Dataframe fazendo a transposição das linhas das partidas para uma linha única.
df_faltas_temp = df_faltas.copy(deep=True)
df_faltas_temp.head()

In [33]:
#Criando a coluna Mandante no DataFrame de faltas temporário e atribuindo um valor aleatório
df_faltas_temp['mandante'] = 3
#df_faltas_temp

In [34]:
#Percorrendo o Data Frame de Faltas
for linha in range(len(df_faltas_temp)):

    #Coletando o Id 
    id=df_faltas_temp.iloc[linha,0].astype(str)

    #Buscando o mandante do confronto pelo ID no Data Frame dos jogos
    mandante_jogos = df_jogos[['ID','mandante']].query('ID=='+id)
    mandante_jogos = mandante_jogos.iloc[0,1]

    #Coletando o time no Data Frame dos faltas
    mandante_faltas = df_faltas_temp.iloc[linha,2]
    
    #Identificando se o clube é mandante, caso seja o mesmo clube significa que o clube é mandante caso contrário o clube é
    #visitante
    
    if mandante_faltas==mandante_jogos:
        #Atribui o valor 1 na Coluna Mandante
        df_faltas_temp['mandante'].iloc[linha] = 1
        
    else:
        df_faltas_temp['mandante'].iloc[linha] = 0


C:\Users\Diogo\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
#Conferindo se a coluna de mandante foi atribuida corretamente
df_jogos[df_jogos.ID==8011]

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
8010,8011,37,9/11/2022,21:29,Juventude,Flamengo,4-2-3-1,4-3-3,C. Juarez Roth,D. Silvestre Júnior,-,Alfredo Jaconi,2,2,RS,RJ


In [36]:
#Criando um DataFrame com os registros do DataFrame de cartões amarelos em um DataFrame com as linhas transpostas
df_faltas_tratados_temp = df_faltas_temp
df_faltas_tratados_temp= df_faltas_tratados_temp[['partida_id','clube','cartao_amarelo','cartao_vermelho','passes','faltas','mandante']]

mask = df_faltas_tratados_temp['mandante'] == 1

df_faltas_tratados_temp = df_faltas_tratados_temp.drop(columns = ['mandante'])

mandante = df_faltas_tratados_temp.loc[mask]
visitante = df_faltas_tratados_temp.loc[~mask]

colunas_mandante = ['partida_id','clube','cartao_amarelo','cartao_vermelho','passes','faltas']
colunas_visitante = ['partida_id','clube','cartao_amarelo','cartao_vermelho','passes','faltas']


mandante.columns = colunas_mandante
visitante.columns = colunas_visitante

df_faltas_tratados = mandante.merge(visitante, on='partida_id')
df_faltas_tratados.head()

,partida_id,clube_x,cartao_amarelo_x,cartao_vermelho_x,passes_x,faltas_x,clube_y,cartao_amarelo_y,cartao_vermelho_y,passes_y,faltas_y
0,1,Guarani,0,0,0,0,Vasco,0,0,0,0
1,2,Athletico-PR,0,0,0,0,Gremio,0,0,0,0
2,3,Flamengo,0,0,0,0,Coritiba,0,0,0,0
3,4,Goias,0,0,0,0,Paysandu,0,0,0,0
4,5,Internacional,0,0,0,0,Ponte Preta,0,0,0,0


In [37]:
#Renomeando as colunas

df_faltas_tratados.rename(columns={'clube_x':'clube_mandante'},inplace=True)
df_faltas_tratados.rename(columns={'clube_y':'clube_visitante'},inplace=True)
df_faltas_tratados.rename(columns={'faltas_x':'faltas_mandante'},inplace=True)
df_faltas_tratados.rename(columns={'faltas_y':'faltas_visitante'},inplace=True)
df_faltas_tratados.rename(columns={'cartao_amarelo_x':'cartao_amarelo_mandante'},inplace=True)
df_faltas_tratados.rename(columns={'cartao_amarelo_y':'cartao_amarelo_visitante'},inplace=True)
df_faltas_tratados.rename(columns={'cartao_vermelho_x':'cartao_vermelho_mandante'},inplace=True)
df_faltas_tratados.rename(columns={'cartao_vermelho_y':'cartao_vermelho_visitante'},inplace=True)
df_faltas_tratados.rename(columns={'passes_x':'passes_mandante'},inplace=True)
df_faltas_tratados.rename(columns={'passes_y':'passes_visitante'},inplace=True)

In [38]:
#Criando a coluna de total de faltas da partida
df_faltas_tratados['total_faltas_partida']=df_faltas_tratados[['faltas_mandante','faltas_visitante']].sum(axis=1)

#Criando a coluna de total de passes da partida
df_faltas_tratados['total_passes_partida']=df_faltas_tratados[['passes_mandante','passes_visitante']].sum(axis=1)

#Criando a coluna de total de cartoes amarelos
df_faltas_tratados['total_cartoes_amarelos']=df_faltas_tratados[['cartao_amarelo_mandante','cartao_amarelo_visitante']].sum(axis=1)

#Criando a coluna de total de cartoes vermelhos
df_faltas_tratados['total_cartoes_vermelhos']=df_faltas_tratados[['cartao_vermelho_mandante','cartao_vermelho_visitante']].sum(axis=1)

#Criando a coluna de total de cartoes da partida
df_faltas_tratados['total_cartoes_partida']=df_faltas_tratados[['total_cartoes_amarelos','total_cartoes_vermelhos']].sum(axis=1)


df_faltas_tratados.tail()

,partida_id,clube_mandante,cartao_amarelo_mandante,cartao_vermelho_mandante,passes_mandante,faltas_mandante,clube_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,passes_visitante,faltas_visitante,total_faltas_partida,total_passes_partida,total_cartoes_amarelos,total_cartoes_vermelhos,total_cartoes_partida
8020,8021,Cuiaba,1,0,505,9,Coritiba,3,1,389,8,17,894,4,1,5
8021,8022,Bragantino,3,0,319,16,Fluminense,2,0,463,15,31,782,5,0,5
8022,8023,Corinthians,3,0,494,11,Atletico-MG,6,0,351,19,30,845,9,0,9
8023,8024,Internacional,2,0,287,13,Palmeiras,3,0,306,17,30,593,5,0,5
8024,8025,Goias,2,0,330,12,Sao Paulo,1,0,591,14,26,921,3,0,3


In [39]:
#Identificando as partidas que não possuem cartões, porém, que não são dados faltantes
df_faltas_tratados[(df_faltas_tratados.total_cartoes_partida==0) & (df_faltas_tratados.total_faltas_partida!=0) & (df_faltas_tratados.total_passes_partida!=0)]
df_faltas_tratados[df_faltas_tratados.partida_id==6828]


,partida_id,clube_mandante,cartao_amarelo_mandante,cartao_vermelho_mandante,passes_mandante,faltas_mandante,clube_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,passes_visitante,faltas_visitante,total_faltas_partida,total_passes_partida,total_cartoes_amarelos,total_cartoes_vermelhos,total_cartoes_partida
4842,4853,Fluminense,0,0,363,17,Gremio,0,0,383,19,36,746,0,0,0
5036,5037,Atletico-MG,0,0,462,12,Cruzeiro,0,0,268,13,25,730,0,0,0
5127,5128,Gremio,0,0,582,6,Sport,0,0,462,8,14,1044,0,0,0
5148,5149,Athletico-PR,0,0,516,6,Sport,0,0,491,11,17,1007,0,0,0
5173,5174,Sport,0,0,481,7,Ponte Preta,0,0,542,16,23,1023,0,0,0
5206,5207,Sao Paulo,0,0,464,10,Internacional,0,0,420,10,20,884,0,0,0
5333,5334,Sao Paulo,0,0,418,7,Atletico-MG,0,0,516,9,16,934,0,0,0
5363,5364,Atletico-MG,0,0,553,7,Chapecoense,0,0,364,11,18,917,0,0,0
5374,5375,America-MG,0,0,317,8,Fluminense,0,0,490,9,17,807,0,0,0
5443,5444,Cruzeiro,0,0,538,9,Flamengo,0,0,336,9,18,874,0,0,0


In [41]:
#Conferindo o DataFrame
df_faltas_tratados_temp.head()

,partida_id,clube,cartao_amarelo,cartao_vermelho,passes,faltas
0,1,Guarani,0,0,0,0
1,1,Vasco,0,0,0,0
2,2,Athletico-PR,0,0,0,0
3,2,Gremio,0,0,0,0
4,3,Flamengo,0,0,0,0


In [44]:
#Salvando o Dataframe em um diretório local
df_faltas_tratados = df_faltas_tratados_temp
df_faltas_tratados.to_csv('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro\\df_faltas_tratados.csv')